In [29]:
import re
from nltk import word_tokenize
from collections import Counter, defaultdict
import math, string

## `regex_repetitions(string)`

Recibe como parámetro un string y determina el número de repeticiones que hay dentro de un `string` utilizando una expresión regular. 


Por ejemplo, con el string `hoooooolaaaa`:

Primero se debe hacer un casting a una lista para que guarde todas las repeticiones que encontró
```python
list(regex_repetitions('hoooooolaaaa'))
```


nos retorna una lista con la letra repetida y cuántas veces se repite
```python
[('o', 6.0), ('a', 4.0)]
```


[Fuente](https://stackoverflow.com/questions/9079797/detect-repetitions-in-string).

In [2]:
def regex_repetitions(string):
   r = re.compile(r"(.+?)\1(.+?)\1+")
   for match in r.finditer(string):
       yield (match.group(1), len(match.group(0))/len(match.group(1)))
    
list(regex_repetitions('hoooooolaaaa'))

[('o', 6.0), ('a', 4.0)]

##  `repetitions(string)`

Recibe un string como parámetro. Utiliza la función `regex_repetitions(string)` para determinar el número de repeticiones en el string.

Y por cada repetición que encuentra es una tupla con el caracter repetido y el número de repeticiones, estos se usan para hacer un nuevo string.

Por ejemplo, con el string hoooooolaaaa y retorna un nuevo string reducido:

```python
test = repetitions('hoooooolaaaa')
print(test)
```
Resultado
```python
ho6la4
```
donde o6 indica que la letra 'o' se repite seis veces y a la 'a' cuatro veces.

In [3]:
def repetitions(string):
    rept_list = list(regex_repetitions(string))
    #new_string = string
    for repetition in rept_list:
        string = string.replace(
                    repetition[0] * int(repetition[1]),
                    repetition[0]
                    +
                    str(int(repetition[1])))
    return string
        

test = repetitions('hoooooolaaaa')
print(test)

ho6la4


### `clean_tweet(tweet)`

Elimina tokens USUARIO, LIGA, es, caracterDeSaltoDeLínea para poder contar las frequencies de las palabras.
    

In [4]:
def clean_tweet(tweet):
    tweet = dict(Counter(tweet))
    tweet.pop('USUARIO', None)
    tweet.pop('LIGA', None)
    tweet.pop('es\n', None)
    tweet.pop('caracterDeSaltoDeLínea', None)
    return tweet

#### Archivos de trabajo. 
* Archivo con todos los tweets (`solo_espanol.txt`).
* Se crea un archivo temporal con 50 tweets para pruebas (`temp.txt`).
* Archivo con tweets cambiando algunos tokens como usuarios o ligas para no mostrarlos (`new_tweets.txt`).

In [5]:
texto_tmp = "temp.txt"
texto = 'solo_espanol.txt'
tweets_limpio = "new_tweets.txt"
tweets_limpio_test = "new_tweets_tmp.txt"

### Tokenizar tweet

Se lee el archivo línea por línea, se tokenizan los tweets y se hacen cambios si el token es un usuario o una dirección url utilizando expresiones regulares y los tweets ya procesados se guardan en un nuevo archivo de texto.

Cada tweet se indexa usando una lista de tuplas.

In [6]:
new_tweets = open(tweets_limpio_test,'w+',encoding='utf-8')

# Expresión regular para validar usuarios de twitter
twitter_username_re = re.compile(r'@([A-Za-z0-9_]+)')

with open(texto_tmp,'r', encoding='utf-8') as file:
    for line in file:
        sentence =line.split(' ')
        for token in range(len(sentence)):
            # Revisa si existen repeticiones en el token
            sentence[token] = repetitions(sentence[token])
            # cambiar un nombre de usuario o una URL por otro token que los identifique
            if sentence[token].find('@') != -1:
                if re.match(twitter_username_re, sentence[token]):
                    sentence[token] = 'USUARIO'
            elif sentence[token].find('http') != -1:
                sentence[token] = 'LIGA'
        
        # Escribe los tweets procesados en un nuevo archivo
        new_tweets.write(' '.join(sentence))
        n_tweet = list(enumerate(word_tokenize(' '.join(sentence))))        
        reverseArr = [x[::-1] for x in n_tweet]
        print((reverseArr))
        print()
        
        
new_tweets.close()

[('USUARIO', 0), ('cuando', 1), ('van', 2), ('a', 3), ('regresar', 4), ('el', 5), ('paquete', 6), ('perdido', 7), ('de', 8), ('nuestros', 9), ('amigos', 10), ('USUARIO', 11), ('USUARIO', 12), ('es', 13)]

[('¿Por', 0), ('qué', 1), ('no', 2), ('aprietan', 3), ('la', 4), ('pasta', 5), ('dental', 6), ('desde', 7), ('abajo', 8), ('?', 9), ('😠😡', 10), ('es', 11)]

[('``', 0), ('No', 1), ('es', 2), ('tu', 3), ('trabajo', 4), ('mejorar', 5), ('un', 6), ('mundo', 7), ('inherentemente', 8), ('caótico', 9), ("''", 10), ('.', 11), ('es', 12)]

[('#', 0), ('HarryStylesConExa', 1), ('Den', 2), ('todo', 3), ('de', 4), ('ustedes', 5), (',', 6), ('yo', 7), ('voy', 8), ('a', 9), ('dar', 10), ('todo', 11), ('y', 12), ('más', 13), ('.', 14), ('❤', 15), ('caracterDeSaltoDeLínea', 16), ('#', 17), ('HarryStylesMexico', 18), ('USUARIO', 19), ('caracterDeSaltoDeLínea', 20), ('(', 21), ('52', 22), (')', 23), ('5543207368', 24), ('es', 25)]

[('Desayuno', 0), ('de', 1), ('go6rdos', 2), ('🙈\U0001f937🏽\u200d♀️', 

In [ ]:
#!/usr/bin/python
#
# Stolen from Ero Carrera
# http://blog.dkbza.org/2007/05/scanning-data-for-entropy-anomalies.html

def range_bytes (): return range(256)
def range_printable(): return (ord(c) for c in string.printable)
def H(data, iterator=range_bytes):
    if not data:
        return 0
    entropy = 0
    for x in iterator():
        p_x = float(data.count(chr(x)))/len(data)
        if p_x > 0:
            entropy += - p_x*math.log(p_x, 2)
    return entropy

In [43]:
freq_words = set()
with open(tweets_limpio_test,'r', encoding='utf-8') as file:
    for i,tweet in enumerate(file):
        print((i))
        for str in word_tokenize(tweet):
            print ("%s:             \t %f" % (str, H(str, range_printable)))
        print()
        #freq_words = freq_words | set(word_tokenize(tweet))
    #print(freq_words)

    #tweets = file.read()

#freq_words1 = clean_tweet(list(word_tokenize(tweets)))
#print(list(freq_words1.keys()))
#'''

0
USUARIO:             	 2.521641
cuando:             	 2.584963
van:             	 1.584963
a:             	 0.000000
regresar:             	 2.155639
el:             	 1.000000
paquete:             	 2.521641
perdido:             	 2.521641
de:             	 1.000000
nuestros:             	 2.750000
amigos:             	 2.584963
USUARIO:             	 2.521641
USUARIO:             	 2.521641
es:             	 1.000000

1
¿Por:             	 1.500000
qué:             	 1.056642
no:             	 1.000000
aprietan:             	 2.750000
la:             	 1.000000
pasta:             	 1.921928
dental:             	 2.584963
desde:             	 1.521928
abajo:             	 1.921928
?:             	 0.000000
😠😡:             	 0.000000
es:             	 1.000000

2
``:             	 0.000000
No:             	 1.000000
es:             	 1.000000
tu:             	 1.000000
trabajo:             	 2.521641
mejorar:             	 2.521641
un:             	 1.000000
mundo:             	 2.32